## 1. Import the libraries and prepare dataset

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('wine.csv')
x = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# Center to mean
x = x - x.mean(axis=0)

## 2a. Getting a sense of SVD

To calculate the SVD, imagine having matrix `A`,

<img src="https://miro.medium.com/max/1400/1*4DncmDEnF9SIYTTrDR0Adw.png" width="650" height="auto" />

Calculate the following eigenvalues and vectors (normalized) from `A^T ⋅ A` and `A ⋅ A^T`,

<img src="https://miro.medium.com/max/1400/1*RG2fiVNyPDr77eS4qsAjLg.jpeg" width="750" height="auto" />

These will be used to form the matrices for `U`, `S` and `V^T`.

<img src="https://miro.medium.com/max/1400/1*xnlAa8E-c63HnMcTRcb5HA.jpeg" width="600" height="auto" />

(Source: [Jonathan Hui, 2019](https://jonathan-hui.medium.com/machine-learning-singular-value-decomposition-svd-principal-component-analysis-pca-1d45e885e491))

Credits: [Jason Brownlee](https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning)

In [49]:
from scipy.linalg import svd

# A = U * S * V^T
A = x
U, S, VT = svd(A)

print(f'A matrix (m × n) = {A.shape}')
print(f'U matrix (m × n) = {U.shape}')
print(f'S matrix (m × n) = {S.shape}')
print(f'V^T matrix (m × n) = {VT.shape}')

A matrix (m × n) = (178, 13)
U matrix (m × n) = (178, 178)
S matrix (m × n) = (13,)
V^T matrix (m × n) = (13, 13)


## 2b. Reformulating SVD

Reformulating it, we can represent it as such,

<img src="https://miro.medium.com/max/1400/1*0LEG-KOZYYYsaXnQxMCkXA.gif" width="500" height="auto" />

And each vectors can be formulated as,

<img src="https://miro.medium.com/max/1400/1*KGcqnL20ihPN4RDyLhQzXA.jpeg" width="600" height="auto" />

Giving this linear combination,

<img src="https://miro.medium.com/max/1400/1*0n2-o06c_j42d0MJo7igYQ.gif" width="500" height="auto" />

(Source: [Jonathan Hui, 2019](https://jonathan-hui.medium.com/machine-learning-singular-value-decomposition-svd-principal-component-analysis-pca-1d45e885e491))

In [50]:
from scipy.linalg import svd

# Reconstruct the matrix and gives back the same, since the last element is 0
# This calculates the same way as 2a.
# Convert U(m × m) . sigma(n × n) . V^T(n × n)
# Into    U(m × m) . sigma(m × n) . V^T(n × n)
# Create the m × n matrix
sigma = np.zeros(A.shape)
# Populate the n × n matrix
sigma[:A.shape[1], :A.shape[1]] = np.diag(S)
# Reconstruct matrix
B = U.dot(sigma.dot(VT))
print('\nReconstructed matrix B,\n')
print(f'A(1) = {A[0]}\n')
print(f'B(1) = {B[0]}\n')
print(f'A(n) = {A[-1]}\n')
print(f'B(n) = {B[-1]}\n')


Reconstructed matrix B,

A(1) = [ 1.22938202e+00 -6.26348315e-01  6.34831461e-02 -3.89494382e+00
  2.72584270e+01  5.04887640e-01  1.03073034e+00 -8.18539326e-02
  6.99101124e-01  5.81910118e-01  8.25505618e-02  1.30831461e+00
  3.18106742e+02]

B(1) = [ 1.22938202e+00 -6.26348315e-01  6.34831461e-02 -3.89494382e+00
  2.72584270e+01  5.04887640e-01  1.03073034e+00 -8.18539326e-02
  6.99101124e-01  5.81910118e-01  8.25505618e-02  1.30831461e+00
  3.18106742e+02]

A(n) = [   1.12938202    1.76365169    0.37348315    5.00505618   -3.74157303
   -0.24511236   -1.26926966    0.19814607   -0.24089888    4.14191012
   -0.34744944   -1.01168539 -186.89325843]

B(n) = [   1.12938202    1.76365169    0.37348315    5.00505618   -3.74157303
   -0.24511236   -1.26926966    0.19814607   -0.24089888    4.14191012
   -0.34744944   -1.01168539 -186.89325843]



## 2c. Pseudo-Inverse SVD

For a linear equation system, compute the inverse of a square matrix `A` to solve `x` where,

<img src="https://miro.medium.com/max/1400/1*a1inq-_XL9WHTCsxzHpamQ.jpeg" width="600" height="auto" />

But as not all matrices are invertible, it will be unlikely to find an exact solution. Hence to find the best-fit solution, compute the following pseudoinverse `A+` which minimizes the least square error,

<img src="https://miro.medium.com/max/1400/1*aQ2MqySUZnIbCIrxfH1G8Q.png" width="600" height="auto" />

Which solution for `x` can be estimated as,

<img src="https://miro.medium.com/max/1400/1*lF1z-LodZHA3834kseswYw.jpeg" width="600" height="auto" />

Leading to,

<img src="https://miro.medium.com/max/1400/1*ClzObIIjZyQDb9svX4FjjQ.jpeg" width="600" height="auto" />

<img src="https://miro.medium.com/max/1400/1*mu9Z_NVYd_3CXwhbERVB8Q.jpeg" width="600" height="auto" />

For illustration purposes (see how the `D+` is constructed for none invertible matrices),

<img src="https://miro.medium.com/max/1400/1*xxatolWVNPjMCUEEWLfvyg.jpeg" width="600" height="auto" />

(Source: [Jonathan Hui, 2019](https://jonathan-hui.medium.com/machine-learning-singular-value-decomposition-svd-principal-component-analysis-pca-1d45e885e491))

In [51]:
from scipy.linalg import svd

# B = A+
A_PI = np.linalg.pinv(A)

print(f'A matrix (m × n) = {A.shape}')
print(f'A+ matrix (m × n) = {A_PI.shape}')
print('\nReconstructed matrix A+,\n')
print(f'A(1) = {A[0]}\n')
print(f'A+(1) = {A_PI[:,0].flatten()}\n')
print(f'A(n) = {A[-1]}\n')
print(f'A+(n) = {A_PI[:,-1].flatten()}\n')

A matrix (m × n) = (178, 13)
A+ matrix (m × n) = (13, 178)

Reconstructed matrix A+,

A(1) = [ 1.22938202e+00 -6.26348315e-01  6.34831461e-02 -3.89494382e+00
  2.72584270e+01  5.04887640e-01  1.03073034e+00 -8.18539326e-02
  6.99101124e-01  5.81910118e-01  8.25505618e-02  1.30831461e+00
  3.18106742e+02]

A+(1) = [ 1.18824971e-02 -3.65460602e-03  2.11654843e-03 -1.96932056e-03
  8.76744001e-04 -1.62499208e-02 -5.95520631e-03  5.27644561e-02
  7.05563663e-03  1.50109879e-03 -1.84029302e-02  3.53394217e-02
 -2.69207210e-05]

A(n) = [   1.12938202    1.76365169    0.37348315    5.00505618   -3.74157303
   -0.24511236   -1.26926966    0.19814607   -0.24089888    4.14191012
   -0.34744944   -1.01168539 -186.89325843]

A+(n) = [ 1.72162774e-02  1.30842923e-03  1.67310309e-02  1.23266371e-03
 -1.83457782e-04  2.03152471e-02 -1.38321782e-02  2.50984340e-02
  7.13578657e-03  1.08775708e-03  6.88656512e-03 -3.23671167e-03
 -3.36088796e-05]



In [52]:
# This calculates the same way as 2c.
# Populate D with n × n diagonal matrix based on A's m × n
D = np.zeros(A.shape)
D[:A.shape[1], :A.shape[1]] = np.diag(1.0 / S)
# Calculate pseudoinverse
A_PI = VT.T.dot(D.T).dot(U.T)

print(f'A matrix (m × n) = {A.shape}')
print(f'A+ matrix (m × n) = {A_PI.shape}')

A matrix (m × n) = (178, 13)
A+ matrix (m × n) = (13, 178)


## 3. Dimensionality Reduction

[Read here on "difference between SVD and PCA"](https://stats.stackexchange.com/questions/239481/difference-between-scikit-learn-implementations-of-pca-and-truncatedsvd) 

In [53]:
from numpy import array
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=2)
svd.fit(A)

T = svd.transform(A)
print(f'T matrix (m × n) = {T.shape}')
print(f'\nT(1) = {T[0]}\n')
print(f'T(n) = {T[-1]}\n')

T matrix (m × n) = (178, 2)

T(1) = [318.56297929  21.49213073]

T(n) = [-186.94319027   -0.2133308 ]



In [54]:
from numpy import array
from numpy import diag
from numpy import zeros
from scipy.linalg import svd

# T = V^k . A
# T = U . Sigma^k
components_n = 2

sigma_k = sigma[:, :components_n]
VT_k = VT[:components_n, :]

print(f'V^T matrix (m × n) = {VT.shape}')
print(f'V^Tk matrix (m × n) = {VT_k.shape}')
print(f'S matrix (m × n) = {sigma.shape}')
print(f'Sk matrix (m × n) = {sigma_k.shape}')

# Reconstruct
B_k = U.dot(sigma_k.dot(VT_k))

print(f'\nA matrix (m × n) = {A.shape}')
print(f'B matrix (m × n) = {B_k.shape}')
print('\nReconstructed matrix B,\n')
print(f'A(1) = {A[0]}\n')
print(f'Bk(1) = {B_k[0]}\n')
print(f'A(n) = {A[-1]}\n')
print(f'Bk(n) = {B_k[-1]}\n')

T_k = U.dot(sigma_k)
print('T = U . Sk (m × n): ', T_k.shape)
print(f'T(1) = {T_k[0]}')
print(f'T(n) = {T_k[-1]}\n')

T_k = A.dot(VT_k.T)
print('T = A . V^Tk (m × n): ', T_k.shape)
print(f'T(1) = {T_k[0]}')
print(f'T(n) = {T_k[-1]}')

V^T matrix (m × n) = (13, 13)
V^Tk matrix (m × n) = (2, 13)
S matrix (m × n) = (178, 13)
Sk matrix (m × n) = (178, 2)

A matrix (m × n) = (178, 13)
B matrix (m × n) = (178, 13)

Reconstructed matrix B,

A(1) = [ 1.22938202e+00 -6.26348315e-01  6.34831461e-02 -3.89494382e+00
  2.72584270e+01  5.04887640e-01  1.03073034e+00 -8.18539326e-02
  6.99101124e-01  5.81910118e-01  8.25505618e-02  1.30831461e+00
  3.18106742e+02]

Bk(1) = [ 5.54444075e-01 -1.70631193e-01  1.60817995e-01 -9.19628125e-01
  2.71701216e+01  3.34192369e-01  4.98165648e-01 -6.83214917e-02
  2.98886635e-01  1.06588043e+00  3.82038649e-02  1.49442298e-01
  3.18124576e+02]

A(n) = [   1.12938202    1.76365169    0.37348315    5.00505618   -3.74157303
   -0.24511236   -1.26926966    0.19814607   -0.24089888    4.14191012
   -0.34744944   -1.01168539 -186.89325843]

Bk(n) = [-3.10444964e-01  1.26851497e-01 -3.74162773e-02  8.67625150e-01
 -3.55349322e+00 -1.85229215e-01 -2.92982814e-01  2.32991653e-02
 -1.13347129e-01 -4.38

## 4. Finding the top features

Note that you can also compress images using SVD - [read here](https://rdsquare.github.io/Image-Compression-SVD-Python/).

In [72]:
from scipy.linalg import svd, eigvals

# Get the eigenvalues
ev = np.sqrt(eigvals(np.dot(x.T, x)))
ev_map = list(map(lambda v: v[0], sorted(enumerate(ev), key=lambda v: v[1], reverse=True)))

U, S, VT = svd(x, full_matrices=0)

cov = np.dot(np.dot(VT.T, np.diag(S**2)), VT)
cov = cov / (x.shape[0] - 1)

features_n = 5

print('Selecting best 5 features\n', S[0:features_n], [dataset.columns[ev_map[i]] for i in range(features_n)], '\n')

print('Gives us related covariance vectors (relationship amongst variables)\n', cov[0:5])

Selecting best 5 features
 [4190.31224906  174.75337527   40.8723149    29.72269526   14.74807124] ['Alcohol', 'Malic_Acid', 'Ash', 'Ash_Alcanity', 'Magnesium'] 

Gives us related covariance vectors (relationship amongst variables)
 [[ 6.59062328e-01  8.56113090e-02  4.71151590e-02 -8.41092903e-01
   3.13987812e+00  1.46887218e-01  1.92033222e-01 -1.57542595e-02
   6.35175205e-02  1.02828254e+00 -1.33134432e-02  4.16978226e-02
   1.64567185e+02]
 [ 8.56113090e-02  1.24801540e+00  5.02770393e-02  1.07633171e+00
  -8.70779534e-01 -2.34337723e-01 -4.58630366e-01  4.07333619e-02
  -1.41146982e-01  6.44838183e-01 -1.43325638e-01 -2.92447483e-01
  -6.75488666e+01]
 [ 4.71151590e-02  5.02770393e-02  7.52646353e-02  4.06208278e-01
   1.12293658e+00  2.21455913e-02  3.15347299e-02  6.35847140e-03
   1.51557799e-03  1.64654327e-01 -4.68215451e-03  7.61835841e-04
   1.93197391e+01]
 [-8.41092903e-01  1.07633171e+00  4.06208278e-01  1.11526862e+01
  -3.97476036e+00 -6.71149146e-01 -1.17208281e+00 

## 5. Another Sample Example

For example, we have a matrix contains the return of stock yields traded by different investors.

<img src="https://miro.medium.com/max/1400/1*FJXUrl22HERjCUe2dR42mA.gif" width="600" height="auto" />

As a fund manager, you want to identify the combination of stocks and investors that have the largest yields.

(Source: [Jonathan Hui, 2019](https://jonathan-hui.medium.com/machine-learning-singular-value-decomposition-svd-principal-component-analysis-pca-1d45e885e491))

<img src="https://i.stack.imgur.com/8JGdq.png" width="200" height="auto" />

(Source: [@jeffery_the_wind, 2013](https://math.stackexchange.com/questions/479918/trying-to-check-cov-matrix-calculation-from-svd-using-numpy))

In [86]:
stocks = ['GOOG', 'AMZN', 'FB', 'TESLA', 'APPL']
stock_yields = np.array([
    [0.8,  0.2,  0.1, 0.04, 0.3],
    [0,    0,    0.2, 0.8,  0.25],
    [0.75,  0.67, 0,   0.04, 0.1],
    [0.75, 0.2,  0.3, 0.15, 0.4],
    [0,    0.3,  0.6, 0.02, 0],
    [0.1,  0,  0.2, 0.04, 0.15],
    [0.3,    0,    0.4, 0.8,  0.2],
    [0.05,  0.7, 0,   0, 0.5],
    [0.62, 0,  0.3, 0.8, 0.6],
    [0,    0.3,  0.6, 0.1, 0.8],
    [0.8,    0.02,  0.3, 0.02, 0],
    [0.02,    0.01,  0.6, 0.025, 0.5],
    [0.95, 0.8,  0.3, 0.8, 0.8],
    [0.08,    0,  0.6, 0.01, 0.9],
    [0.02,    0,  0.35, 0.65, 0.5],
    [0.5,    0.1,  0.65, 0.1, 0.25],
    [0.01,    0.5,  0.55, 0.3, 0.15]  
])

cov = np.cov(stock_yields.T)

print(f"""Using numpy covariance
    cov shape: {cov.shape}
    cov[1]: {cov[0]}
    cov[n]: {cov[-1]}
""")

stock_yields_mean = stock_yields - stock_yields.mean(axis=0)

U, S, VT = svd(stock_yields_mean, full_matrices=0)

cov = np.dot(np.dot(VT.T, np.diag(S**2)), VT)
cov = cov / (stock_yields.shape[0] - 1)

print(f"""Using SVD covariance
    cov shape: {cov.shape}
    cov[1]: {cov[0]}
    cov[n]: {cov[-1]}
""")

# Get eigenvalues
ev = np.sqrt(eigvals(np.dot(stock_yields_mean.T, stock_yields_mean)))
ev_map = list(map(lambda v: v[0], sorted(enumerate(ev), key=lambda v: v[1], reverse=True)))
features_n = 2

print('Displaying stocks and their std. covariance\n', S, [stocks[ev_map[i]] for i in range(len(stocks))])

print('\nDisplaying stocks covariance matrix\n', cov)

print('\nSelecting best stocks with higher combination yields\n', S[0:features_n], [stocks[ev_map[i]] for i in range(features_n)])

print('\nGives us related covariance vectors (relationship amongst variables)\n', cov[:,0:features_n])

Using numpy covariance
    cov shape: (5, 5)
    cov[1]: [ 0.13167794  0.02458787 -0.03080147  0.01061158 -0.00620037]
    cov[n]: [-0.00620037  0.00696324  0.01202206  0.01818566  0.07878676]

Using SVD covariance
    cov shape: (5, 5)
    cov[1]: [ 0.13167794  0.02458787 -0.03080147  0.01061158 -0.00620037]
    cov[n]: [-0.00620037  0.00696324  0.01202206  0.01818566  0.07878676]

Displaying stocks and their std. covariance
 [1.57553585 1.39921259 1.14559555 0.9894796  0.67449509] ['GOOG', 'APPL', 'TESLA', 'FB', 'AMZN']

Displaying stocks covariance matrix
 [[ 0.13167794  0.02458787 -0.03080147  0.01061158 -0.00620037]
 [ 0.02458787  0.07812426 -0.02002206 -0.00925129  0.00696324]
 [-0.03080147 -0.02002206  0.04652574 -0.00608548  0.01202206]
 [ 0.01061158 -0.00925129 -0.00608548  0.11404228  0.01818566]
 [-0.00620037  0.00696324  0.01202206  0.01818566  0.07878676]]

Selecting best stocks with higher combination yields
 [1.57553585 1.39921259] ['GOOG', 'APPL']

Gives us related cova